In [54]:
# Purpose of this doc is to be able to import any listsource data and format it in a way such that complete names, companies, etc. are all 
# parsed and ready for proper skip-tracing
import pandas as pd
import os.path
from upload_to_drive import export_to_folder

In [55]:
filename = "sample"
raw = pd.read_csv(f'./data/{filename}/{filename}.csv').drop(columns=['Owner Address Combined', 'Property Combined'])

In [56]:
raw.columns

Index(['OWNER 1 LABEL NAME', 'OWNER 1 LAST NAME', 'OWNER 1 FIRST NAME',
       'OWNER 1 MIDDLE NAME', 'OWNER 1 SUFFIX', 'OWNER 2 LABEL NAME',
       'OWNER 2 LAST NAME', 'OWNER 2 FIRST NAME', 'OWNER 2 MIDDLE NAME',
       'OWNER 2 SUFFIX', 'OWNER CARE OF NAME', 'MAIL ADDRESS', 'MAIL CITY',
       'MAIL STATE', 'MAIL ZIP CODE', 'MAIL ZIP+4', 'MAIL ZIP/ZIP+4',
       'MAIL CARRIER ROUTE', 'MAIL COUNTRY', 'PROPERTY ADDRESS',
       'PROPERTY HOUSE NUMBER', 'PROPERTY HOUSE NUMBER PREFIX',
       'PROPERTY HOUSE NUMBER SUFFIX', 'PROPERTY HOUSE NUMBER 2',
       'PROPERTY PRE DIRECTION', 'PROPERTY STREET NAME',
       'PROPERTY STREET NAME SUFFIX', 'PROPERTY POST DIRECTION',
       'PROPERTY UNIT NUMBER', 'PROPERTY CITY', 'PROPERTY STATE',
       'PROPERTY ZIP CODE', 'PROPERTY ZIP+4', 'PROPERTY ZIP/ZIP+4',
       'PROPERTY CARRIER ROUTE', 'COUNTY', 'UNITS - NUMBER OF'],
      dtype='object')

In [57]:
LISTSOURCE_MAPPING = {
    "OWNER 1 LABEL NAME": "full_name",
    "OWNER 1 LAST NAME": "last_name",
    "OWNER 1 MIDDLE NAME": "middle_name",
    "OWNER 1 FIRST NAME": "first_name",
    "MAIL ADDRESS": "mail_add",
    "MAIL CITY": "mail_city",
    "MAIL STATE": "maill_state",
    "MAIL ZIP CODE": "mail_zip",
    "PROPERTY ADDRESS": "prop_add",
    "PROPERTY UNIT NUMBER": "prop_unit",
    "PROPERTY CITY": "prop_city",
    "PROPERTY STATE": "prop_state",
    "PROPERTY ZIP CODE": "prop_zip",
    "COUNTY": "prop_county",
    "UNITS - NUMBER OF": "prop_size"
}
ENTITY_NAMES = "|".join([" Inc", " Trust ", "L L C", " LLC", " Corp", " Home ", " Co "])
mapped_cols = raw[LISTSOURCE_MAPPING.keys()].rename(LISTSOURCE_MAPPING, axis=1)

In [58]:
mask = (
    (mapped_cols.first_name.str.len() < 2) |
    (mapped_cols.last_name.str.len() < 2)  | 
    (mapped_cols.full_name.str.contains(ENTITY_NAMES, na=True, regex=True))
)

In [59]:
incomplete = mapped_cols[mask]
complete = mapped_cols[~mask]

In [72]:
len(incomplete), len(complete)

(2, 417)

In [61]:
clean_file = f"./data/{filename}/clean.csv"
dirty_file = f"./data/{filename}/incomplete.csv"
complete.to_csv(clean_file)
incomplete.to_csv(dirty_file)

In [65]:
export_to_folder(f"try_again_munge/{filename}", clean_file, "text/csv")
export_to_folder(f"try_again_munge/{filename}", dirty_file, "text/csv")

TypeError: export_to_folder() takes 2 positional arguments but 3 were given